In [192]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from transformers import AutoModel, AutoTokenizer, BertTokenizer

In [195]:
train_df = pd.read_csv('./dataset/train/train_revised.csv')
test_df = pd.read_csv('./dataset/test/test_data.csv')
MODEL_NAME = "klue/roberta-large"
# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [179]:
train_df.head(3)

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree


In [180]:
train_df['sentence'][0]

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [181]:
sc_dict_train = {}
for i in range(32428):
    string = train_df['sentence'][i]
    for char in string:
        if not char.isalnum():
            try:
                sc_dict_train[char] += 1
            except:
                sc_dict_train[char] = 1

In [182]:
sc_dict_train

{'〈': 185,
 '〉': 191,
 ' ': 636954,
 '《': 934,
 '》': 926,
 '.': 36161,
 '·': 2938,
 '(': 19195,
 ')': 19170,
 '‘': 2186,
 '’': 2185,
 ':': 672,
 ',': 37584,
 '~': 3156,
 "'": 6132,
 '-': 3023,
 '“': 1988,
 '”': 1964,
 '"': 5390,
 '㈜': 209,
 '‧': 138,
 '↔': 2,
 '▲': 242,
 '%': 981,
 '/': 244,
 '&': 218,
 '!': 134,
 '–': 86,
 ';': 60,
 '=': 15,
 '→': 21,
 '€': 29,
 '△': 203,
 '*': 15,
 '｢': 10,
 '｣': 10,
 '★': 3,
 '>': 2,
 '․': 104,
 '「': 98,
 '」': 98,
 '・': 36,
 'Ⓐ': 6,
 '•': 8,
 '☎': 24,
 '[': 349,
 ']': 344,
 '㎞': 8,
 '☆': 2,
 '`': 35,
 '※': 4,
 '|': 36,
 '『': 27,
 '』': 27,
 '。': 2,
 '˘': 2,
 '，': 4,
 '＇': 1,
 '∼': 34,
 '㎡': 12,
 '́': 2,
 '：': 6,
 '⸱': 3,
 '_': 2,
 '∞': 3,
 '♥': 5,
 '･': 11,
 '⋅': 2,
 '⟪': 7,
 '⟫': 7,
 '～': 4,
 '£': 7,
 '{': 3,
 '}': 3,
 '°': 2,
 '˹': 2,
 '˼': 2,
 '▴': 3,
 '℃': 2,
 '㎏': 2,
 '（': 3,
 '）': 2,
 '∙': 12,
 '♡': 2,
 '$': 8,
 '㎖': 1,
 '≪': 3,
 '≫': 3,
 '□': 1,
 '·': 1,
 '\u200e': 2,
 '☏': 2,
 '⁺': 1,
 '®': 3,
 '、': 1,
 '？': 1,
 '―': 3,
 '○': 5,
 '／': 3,
 '़'

In [183]:
sc_dict_test = {}
for i in range(7765):
    string = test_df['sentence'][i]
    for char in string:
        if not char.isalnum():
            try:
                sc_dict_test[char] += 1
            except:
                sc_dict_test[char] = 1

In [184]:
sc_dict_test

{' ': 151075,
 "'": 1685,
 '.': 8655,
 '‘': 806,
 '’': 810,
 ':': 84,
 '(': 2831,
 ')': 2831,
 ',': 6975,
 '"': 1682,
 '-': 420,
 '▲': 120,
 '“': 1441,
 '”': 1442,
 '~': 180,
 '㈜': 42,
 '·': 688,
 '/': 67,
 '《': 197,
 '》': 196,
 '!': 37,
 '。': 1,
 '㎡': 5,
 '%': 395,
 '‧': 43,
 '「': 32,
 '」': 32,
 '『': 10,
 '』': 10,
 '&': 42,
 '〈': 39,
 '〉': 39,
 '☎': 4,
 '･': 9,
 '․': 29,
 '–': 8,
 '△': 29,
 '´': 1,
 '∙': 3,
 '*': 5,
 '€': 3,
 '$': 5,
 '[': 49,
 ']': 47,
 '∼': 18,
 '|': 2,
 '・': 5,
 '㎞': 4,
 '㎏': 3,
 '☏': 2,
 '―': 2,
 '_': 5,
 '≪': 2,
 '≫': 2,
 '（': 1,
 '）': 1,
 '`': 4,
 '→': 9,
 '▴': 6,
 '：': 3,
 '～': 4,
 '=': 2,
 '⋅': 1,
 '｢': 1,
 '｣': 1,
 '{': 3,
 '}': 3,
 '⟪': 1,
 '⟫': 1,
 '●': 1,
 '▶': 6,
 ';': 2,
 '£': 1,
 '％': 2,
 '±': 1}

In [185]:
sc_dict_all = sc_dict_train.copy()
for i in range(7765):
    string = test_df['sentence'][i]
    for char in string:
        if not char.isalnum():
            try:
                sc_dict_all[char] += 1
            except:
                sc_dict_all[char] = 1

In [191]:
all_keys = sc_dict_all.keys()
for k in all_keys:
    print(k,end=' ')

〈 〉   《 》 . · ( ) ‘ ’ : , ~ ' - “ ” " ㈜ ‧ ↔ ▲ % / & ! – ; = → € △ * ｢ ｣ ★ > ․ 「 」 ・ Ⓐ • ☎ [ ] ㎞ ☆ ` ※ | 『 』 。 ˘ ， ＇ ∼ ㎡ ́ ： ⸱ _ ∞ ♥ ･ ⋅ ⟪ ⟫ ～ £ { } ° ˹ ˼ ▴ ℃ ㎏ （ ） ∙ ♡ $ ㎖ ≪ ≫ □ · ‎ ☏ ⁺ ® 、 ？ ― ○ ／ ़ ी ु ् ा ㈔ ▵ « » × — ％ ′ ₫ < ㎜  ㎿ ⌜ ⌟ ´ ● ▶ ± 

In [85]:
only_train = list(sc_dict_train.keys()-sc_dict_test.keys())
only_test = list(sc_dict_test.keys()-sc_dict_train.keys())
print('train에만 있음 \n',only_train)
print('test에만 있음 \n',only_test)

train에만 있음 
 ['\u200e', '⁺', '㈔', '㎿', '⸱', 'ी', '>', '□', '′', '、', '㎜', '℃', '×', 'ा', '\uf09e', '®', '＇', '↔', '，', '／', '»', '㎖', '́', '—', '<', 'Ⓐ', '♡', '·', '़', '？', '˼', 'ु', '★', '₫', '्', '☆', '♥', '«', '∞', '⌟', '▵', '˘', '○', '˹', '⌜', '•', '※', '°']
test에만 있음 
 ['●', '±', '´', '▶']


# in train

## 괄호류
() 《》「」『』 〈〉[] ≪≫（）｢｣ {} ⟪⟫ <> ⌜⌟ ˹˼

## 인용구류
' ‘ ’" “ ”` ′＇

## 문장부호류  
. : , ~ ! % & * = ; ？

## 구분자
·/ ∙ | ・ ⋅ ▴ ⸱ ／·▵•

## 작대기?
- ― _ —

## 도형
▲ ☎ △ ☏ □ ↔ Ⓐ ♡ ★ ☆ ♥○ ※

## 기호/단위
㈜ 。㎡ ･ € $ ㎞ ㎏ → £ ㈔ ㎿ ㎜ ℃® ㎖ ∞°

## 이건뭐지..
‧ 。․ – ∼ ：～ ％ \u200e ⁺ ी × 'ा \uf09e ，»  '́ '़   'ु ₫ '् « ˘

---

# in test

## 괄호류
() 《》「」『』 〈〉[] ≪≫（）｢｣ {} ⟪⟫

## 인용구류
' ‘ ’" “ ”´`

## 문장부호류
. : , ~ ! % & * = ;

## 구분자
·/ ∙ | ・ ⋅ ▴

## 작대기?
- ― _

## 도형
▲ ☎ △ ☏ ● ▶

## 기호/단위
㈜ 。㎡ ･ € $ ㎞ ㎏ → £ ±

## 이건뭐지..
‧ 。․ – ∼ ：～ ％

In [209]:
unk_list = []
for key in all_keys:
    text = key
    tokenized_text = tokenizer.tokenize(text)  # 입력된 것을 tokenize하는 함수
    if tokenized_text:
        if tokenized_text[0] == "[UNK]":
            print(key)
            unk_list.append(key)

↔
–
€
Ⓐ
☎
。
˘
＇
́
⸱
∞
･
⋅
⟪
⟫
～
£
°
˹
˼
▴
（
）
·
☏
⁺
®
、
？
／
़
ी
ु
्
ा
㈔
▵
«
»
′
₫
㎿
⌜
⌟
±


In [205]:
unk_list

['↔',
 '–',
 '€',
 'Ⓐ',
 '☎',
 '。',
 '˘',
 '＇',
 '́',
 '⸱',
 '∞',
 '･',
 '⋅',
 '⟪',
 '⟫',
 '～',
 '£',
 '°',
 '˹',
 '˼',
 '▴',
 '（',
 '）',
 '·',
 '☏',
 '⁺',
 '®',
 '、',
 '？',
 '／',
 '़',
 'ी',
 'ु',
 '्',
 'ा',
 '㈔',
 '▵',
 '«',
 '»',
 '′',
 '₫',
 '㎿',
 '⌜',
 '⌟',
 '±']

In [168]:
# 괄호류
bracket = ['(',') ','《','》','「','」','『','』 ','〈','〉','[','] ','≪','≫','（','）','｢','｣ ','{','} ','⟪','⟫ ','<','> ','⌜','⌟ ','˹','˼']
len(bracket)

28

In [169]:
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        if find in df['sentence'][i]:
            print(df['sentence'][i])


for ot in bracket[2:]:
    print('\n\n',ot,'--------------------------------------')
    find_sc(train_df,ot)
    



 《 --------------------------------------
〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음을 먹었던 적이 있었습니다. 잘하려고 애를 쓰는데 비난과 비판이 쏟아졌습니다. 머리가 빠지고 너무 힘들었습니다”라고 말하며 최진실의 죽음에 대해 안타까움을 표현했다.
대한민국에서는 2008년 대원미디어㈜에서 수입하여 자사인 대원방송㈜에서 더빙 작업을 진행하여 《짱구는 못말려 극장판: 부리부리 3분 대작전》이라는 이름으로 방영되었다.
《천녀유혼》(The Ethereal Spirit of a Beauty)은 1987년 개봉한 홍콩의 로맨틱 코미디 호러 영화로 제작자는 서극이며, 영화감독은 정소동, 주연은 장국영, 왕조현, 우마이다.
민 리(1977년 6월 27일 ~)는 베트남계 캐나다인 게임 개발자로, 1999년 제스 클리프와 함께 《하프라이프》의 유명 모드인 《카운터스트라이크》를 제작했다.
그리고 트랙스는 멤버 김정모)가 MBC 《일요일 일요일 밤에 인기코너 오빠밴드 정규 멤버로 투입되면서 "트랜스"라는 별명을 얻었다.
’라고 기술하여, 동예의 위치를 지금의 강릉시 일대로 비정하고 있다. 《동사강목》에서는 ‘고구려 태조왕이 동옥저를 취하였는데, 그 땅이 동쪽으로는 창해에 이르렀다 하였으니, 동옥저의 땅은 지금의 함경남도인 것이다.
1938년 그리스 국립 아테네 음악원에 입학하여 1941년 수료한 뒤 《카발레리아 루스티카나 (Cavalleria Rusticana)》의 산투차 역으로 데뷔하였다.
《Riding with the King》은 2000년 발매된 에릭 클랩튼과 B.B. 킹의 블루스 음반이다.
투개월 정식 데뷔 음반을 준비하던 중 도대윤이 고등

 》 --------------------------------------
〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음을 먹었던 적이 있었습니다. 잘하려고 애를 쓰는데 비난과 비판이 쏟아졌습니다. 머리가 빠지고 너무 힘들었습니다”라고 말하며 최진실의 죽음에 대해 안타까움을 표현했다.
대한민국에서는 2008년 대원미디어㈜에서 수입하여 자사인 대원방송㈜에서 더빙 작업을 진행하여 《짱구는 못말려 극장판: 부리부리 3분 대작전》이라는 이름으로 방영되었다.
《천녀유혼》(The Ethereal Spirit of a Beauty)은 1987년 개봉한 홍콩의 로맨틱 코미디 호러 영화로 제작자는 서극이며, 영화감독은 정소동, 주연은 장국영, 왕조현, 우마이다.
민 리(1977년 6월 27일 ~)는 베트남계 캐나다인 게임 개발자로, 1999년 제스 클리프와 함께 《하프라이프》의 유명 모드인 《카운터스트라이크》를 제작했다.
’라고 기술하여, 동예의 위치를 지금의 강릉시 일대로 비정하고 있다. 《동사강목》에서는 ‘고구려 태조왕이 동옥저를 취하였는데, 그 땅이 동쪽으로는 창해에 이르렀다 하였으니, 동옥저의 땅은 지금의 함경남도인 것이다.
1938년 그리스 국립 아테네 음악원에 입학하여 1941년 수료한 뒤 《카발레리아 루스티카나 (Cavalleria Rusticana)》의 산투차 역으로 데뷔하였다.
《Riding with the King》은 2000년 발매된 에릭 클랩튼과 B.B. 킹의 블루스 음반이다.
투개월 정식 데뷔 음반을 준비하던 중 도대윤이 고등학교 학업을 마치기 위하여 당분간 미국으로 떠나게 되자 그 동안 김예림은 자신의 첫 솔로 앨범인 《A Voice》를 발표하고 개인활동을 

인천항만공사(사장 남봉현)는 인천지방중소벤처기업청을 비롯해 수도권매립지관리공사, 한국환경공단과 함께 인천지방중소벤처기업청 대강당에서「찾아가는 공공기관 1:1 구매상담회」를 개최했다고 밝혔다.
한국선급(KR, 회장 이정기)과 현대LNG해운(주)(HLS, 사장 이규봉)은 2일 현대LNG해운 부산사무소에서 양사 관계자가 참석한 가운데「선박 사이버보안 규칙 적용, 검증 및 발전을 위한 공동연구」양해각서(MOU)를 체결했다고 밝혔다.
세계야구소프트볼연맹이 주최하고 기장군과 대한야구소프트볼협회가 공동 주관하는 「제29회 WBSC 기장 세계청소년야구선수권대회(18세 이하)」는 이번 달 30일부터 9월 8일까지 10일간 기장-현대차 드림볼파크에서 개최된다.
영암군(군수 전동평)은 2년 연속「대한민국 책 읽는 지자체 대상」에 선정되어 11일 오후 3시 서울 영등포구 KR컨벤션웨딩에서 국회 문화체육관광위원회 위원장 상을 수상했다.
지난해 2월 자유한국당 김진태 등 3인 의원의 '5.18민주화운동 망언' 사건 이후 세 의원의 윤리위 제소를 적극적으로 이끌었으며, 5.18민주화운동 왜곡·폄훼를 처벌하는 내용의 「5.18민주유공자 예우 및 지원에 관한 법률」을 대표발의한 바 있다.
팬의 사이에서는 「아저씨(오얏상)」라고 불리고 있지만, 이것은 극중에서 사용된 통칭이 아니고, 혼고 타케시를 연기한 후지오카 히로시의 독특한 발음으로 「아저씨」가 그렇게 들리는 것에서, 후년 팬 사이에서 애칭으로서 정착한 것.
경기 포천시(시장 박윤국)는 내년 1월 27일까지 국민 관광지 산정호수에서 낭만적인 겨울밤을 선사하는「2019 산정호수 윈터페스타: 별빛공원 달빛거리」를 개최한다.


 」 --------------------------------------
경성대학교(총장 송수건) 미술학과 졸업예정 학생들이 부산 지역의 유망한 청년예술가에게 창작 발표기회를 제공하고 지역 신진예술가를 발굴․육성하는 홍티 예술촌의「청년예술가 발굴 기획전-展」에 참여한다.
채택된 결의안은 지방분권 실현을 위해서는 「지

1918년 윌슨 미국대통령의 전후문제처리방안으로 민족자결주의 원칙이 발표된 후, 국내외에서 한결같이 독립운동이 극에 달하고 독립선언을 하게 되자, 일본은 오랜 무단정치를 버리고 회유정책으로 1920년에 들어 『조선일보』 『동아일보』 『시사신문』의 3대 민간지를 창간케 했으나, 이들 신문들은 창간 직후부터 압수·정간·기자구속·벌금형 등의 수많은 고난을 당했다.
역사학자 구메 구니타케는 『일본외사(日本外史)』와 함께 '극본(劇本)의 일종'라 혹평하고 있다.
아프리카 현대 문학을 이끄는 독보적인 작가 치마만다 응고지 아디치에의 청춘 일기 『아메리카나』. 나이지리아에서 좋은 교육을 받으며 구김 없이 자란 십 대 소년과 소녀가 아메리칸드림으로 각자 미국과 영국으로 떠나 겪는 인종 차별의 순간과 현실적인 경험을 발랄하고 톡톡 튀는 문체로 그려낸 작품이다.
김영삼 정부의 출범 후 민변은 1993년 안기부의 간첩조작사건, 1994년『한국사회의 이해』저자들에 대한 국가보안법 적용 등 문민정부의 이름 아래 자행되는 반민주적 잔재를 고발하고 척결하기 위해 동분서주하면서도, 변화된 사회 속에서 새로운 과제를 인식하고 적극적으로 대응하려 하였다.
이후에는 레이지의 멤버인 이노우에 슌지가 설립한 레코드 회사인 『란티스』(ランティス)로 이적(전속은 아님)했으며 2000년에 결성된 Jam Project의 리더로도 활동하고 있다.
《창문 넘어 도망친 100세 노인》의 저자 요나스 요나손의 신작 『셈을 할 줄 아는 까막눈이 여자』. 스웨덴과 남아공을 배경으로 100세 노인의 삶보다 더 기구하고 황당무계한 까막눈 소녀의 삶을 그려낸 작품이다.
1947년 코헛은 독일 소설가 토마스 만(Thomas Mann)의 중편소설 『베네치아에서의 죽음(Der Tod in Venedig)』에 대해 관심을 가졌고, 1950년 졸업논문으로서 이 소설에 대한 생각들을 정리한 글을 제출하였다.
그 신화는 10세기 말 키나드 2세(재위: 971년–995년) 때 편찬된 『알바 열왕연대기』에서 처음 나타난다.
박남춘 인천

매니저 엡스타인은 이들을 전문적인 활동이 가능하도록 변형시켰고 프로듀서 조지 마틴은 1962년 말 영국에서 막대한 유명세를 얻게 일조한 이들의 첫 히트 싱글 〈Love Me Do〉부터 녹음물을 지휘 및 개발하였다.
오티스 블랙웰(Otis Blackwell, 1931년 2월 16일 ~ 2002년 5월 6일)은 로큰롤에 심대한 영향력을 발휘한 아프리카계 미국인 작곡가, 가수, 피아니스트로 리틀 윌리 존의 〈Fever〉, 제리 리 루이스의 〈Great Balls of Fire〉, 〈Breathless〉, 엘비스 프레슬리의 〈Don't Be Cruel〉, 〈All Shook Up〉, 〈Return to Sender〉, 지미 존스의 〈Handy Man〉의 작곡가로 유명하다.
그 때의 미디 작업으로 〈난 알아요〉, 〈환상속의 그대〉등 서태지와 아이들 1집에 수록될 곡들의 일부가 만들어진다.
2011년 YG 엔터테인먼트의 그룹 2NE1의 앨범 아트웍과 그들의 뮤직 비디오 〈Hate You〉를 연출하여 대중에게도 큰 인기를 얻었다.
프레디 머큐리가 작곡한 곡들로 〈Bohemian Rhapsody〉, 〈We Are the Champions〉, 〈Love of My Life〉, 〈Crazy Little Thing Called Love〉, 〈Somebody to Love〉, 〈Killer Queen〉, 〈Don't Stop Me Now〉 등이 있다.
이 경험을 살려서 써진 곡이 바로 〈Something in the Way〉. 그렇지만 이것은 어디까지나 코베인 본인의 주장으로 너바나의 베이스를 맡은 크리스 노보셀릭은 논박하되 "거기서 있기는 많이 있었습니다. 근데 거기가 원체 뻘밭이라 살지는 못해요. 밀물 썰물이 다 있는 곳인걸. 코베인 고 녀석의 이른바 "수정주의"라는 거지요" 했다.
특히 그중 텍사스 인스트루먼츠(Texas Instruments)가 1972년 저가 계산기 시장을 지배하기 전까지, 1971년 〈파퓰러 일렉트로닉스〉(Popular Electronics)에 이 회사의

1993년에 출연한 KOSE 화장품 〈루쉐리〉의 CM으로 주목을 끌어 다음 1994년, 후지 TV의 게츠9 범위에서 방송된 《너와 함께한 여름》에 헤로인으로 발탁되고 브레이크를 하였다.
"야구 운영들에 특별 고문"으로서 신시내티 레즈와 함께 모건이 공식적 역할을 택한 이후, 2010년 11월 8일 모건이 ESPN 〈일요일 밤 야구〉에 아나운서로서 2011년 시즌을 위하여 돌아오지 않을 것이라고 공고되었다.
이 책의 초판 간행 당시에 시인이자 소설가 박종화는 〈영친왕을 위해 곡하다〉라는 시를 헌정하였고, 원나라의 볼모로 가서 노국대장공주와 정략결혼을 하였지만 고려로 돌아와 고토를 회복한 공민왕에 비유하며 이은이 조국 광복을 위해서 노력하지 않은 것에 대하여 안타까움을 표했다.
이후 나르샤는 계속해서 브라운 아이드 걸스로서 활동을 이어가는 한편,2013년에는 미료와 함께 "M&N"이라는 유닛을 결성, 〈오늘밤〉이라는 디지털 싱글을 발매했으며,TV 조선 3부작 드라마의 여주인공으로 발탁되기도 하였다.
이들은 1882년 6월 6일자로 청나라 마건충(馬建忠)·정여창 등의 임석하에 영국의 윌리스 간에 전문 14조로 된 〈조영수호통상조약〉에 조인하였다.


 [ --------------------------------------
이들은 ['아이즈원' 혼다 히토미, 사쿠라 이어 日 라디오 중단] 기사를 공유하며 "풍악을 울려라" 등 축하를 주고받았다.
《가면라이더 W》에서 가면라이더 스컬로 변신하는 나루미 소우키치([[킷카와 코지]]가 연기)는 [[가면라이더 W의 등장 인물 아키코|나루미 아키코]]의 아버지로, 후토에 나루미 탐정 사무소를 연 인물이며, 나루미 탐정 사무소에서 [[가면라이더 더블 (캐릭터) 쇼타로|히다리 쇼타로]]를 제자로 삼아 그를 가르치고 있었다.
스타드 벨로드롬은 2014년까지 확장 공자를 마칠 계획 ([[UEFA 유로 2016]]을 나중에 개최)이며, 계속적으로 [[올랭피크 드 마르세유]]의 홈경기를 주최할 것이다.
가수 출신 한성호 FNC엔터테인먼

[[2011년]] [[4월 8일]]부터 [[6월 24일]](1기), [[10월 7일]]부터 [[12월 23일]](2기)까지 [[TV 사이타마]] 및 [[전국 독립UHF방송 협의회|U국 계열]]에서 방영. 주요 스태프는 《[[순정 로맨티카]]》의 TV 시리즈와 거의 동일하다.
[[조지아의 국호|이 나라의 국호]]는 [[러시아어]] 이름인 그루지야로 통용되었으나, 조지아 정부는 [[대한민국]]을 비롯한 주변 국가에 러시아어 그루지야 대신 영어 이름 조지아(Georgia)로 자국 국명을 표기해 줄 것을 공식 요청하였으며, [[대한민국 외교부]]는 2010년 이 요청을 적극 수용했다.
1968년 이스탄불로 귀국한 그는 할키 신학교의 교수로 있다가 1969년 [[콘스탄티노폴리스 세계 총대주교 [[콘스탄티노폴리스의 아테나고라스 1세|아테나고라스 1세]]에 의해 [[사제]] 서품을 받았다.
함영준 오뚜기[007310] 회장 장녀로 오뚜기 지분을 1.19% 가진 뮤지컬 배우 함연지 씨의 주식 자산은 작년 말보다 66억원(21.1%) 감소한 246억원이다.
[154] 테레사 메이 총리는 유럽 연합 출국 투표에 이어 보수당 정부가 영국의 기존 연구 개발 프로젝트에 대한 자금을 보호 할 것을 보장했다.
이는 동아일보의 창시자인 송진우, [김성수 (언론인)|김성수] 등이 주동적으로 창당한 한민당이 민국당, 민주당 등으로 바뀌면서 친 민주당적 성향을 띈 것에서 연유한다.
전 독일어 수상 게르하르트 슈뢰더 와 헬무트 슈미트, 독일 재무 장관 볼프강 쇼이블레 프랑스 재무 장관 피에르 모스코비치, 유럽 의회 회장 마틴 슐츠, 구글 회장 인 에릭 슈밋[6] 등이 이 포럼에 참석하였다.
한국수자원공사[K-water] 포항권관리단에서 운영하고 있다.
예명인 k-os(/ˈkeɪ.ɒs/, [[카오스])로 더 잘 알려진 케빈 브레러턴 (Kevin Brereton) 은 [[캐나다|캐나다인] 랩퍼, 가수, 송라이터, 레코드 프로듀서이다.
2016년 2월 2일, SM 엔터테인먼트는 EXO 공식 홈페이지를



 < --------------------------------------
그 밖에 서울 강서구 갑 현역 의원이었던 금태섭의 낙천을 두고 조선일보 같은 보수 언론에선 <금태섭 공천 탈락놓고..


 >  --------------------------------------
27일 오후 10시 48분 과천시청 공식 트위터 계정에 '문재인 대통령 탄핵을 촉구합니다. > 대한민국 청와대'라는 트윗이 올라왔고, 해당 청원으로 통하는 청와대 청원 페이지가 링크로 올라왔다.


 ⌜ --------------------------------------
법제사법위원회 자유한국당 간사 김도읍 국회의원(부산 북구‧강서구 을)이 방송통신심의위원회(이하 방심위)로부터 제출받은 ⌜국가보안법 위반 정보 심의 및 시정조치 현황⌟자료에 따르면 최근 5년(2014~2018)간 인터넷 상 국가보안법 위반으로 시정 조치 된 게시물 수는 총 9,144건에 달하는 것으로 나타났다.


 ⌟  --------------------------------------


 ˹ --------------------------------------
공무원연금공단(이사장 정남준)은 21일 전북 전주시 꽃밭정이노인복지관에서 ˹찾아가는 사랑나눔 따뜻한 겨울나기 문화공연˼을 개최했다.
공무원연금공단(이사장 정남준)은 12일 전북 완주시 삼례 전통시장에서 동절기 ˹재래시장 화재점검 및 안전예방˼ 봉사활동을 실시했다.


 ˼ --------------------------------------
공무원연금공단(이사장 정남준)은 21일 전북 전주시 꽃밭정이노인복지관에서 ˹찾아가는 사랑나눔 따뜻한 겨울나기 문화공연˼을 개최했다.
공무원연금공단(이사장 정남준)은 12일 전북 완주시 삼례 전통시장에서 동절기 ˹재래시장 화재점검 및 안전예방˼ 봉사활동을 실시했다.


In [165]:
# subject, object 확인
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        try:
            if find in df['subject_entity'][i].split('\', \'start_idx')[0].split('word\': \'')[1]:
                print(find,i,": ",df['subject_entity'][i])
        except:
            print('!!!!!!error',find,i,": ",df['subject_entity'][i])


for ot in bracket:
    find_sc(train_df,ot)
    
print('\n----------------------------------------------------------\n')
# subject, object 확인
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        try:
            if find in df['object_entity'][i].split('\', \'start_idx')[0].split('word\': \'')[1]:
                print(find,i,": ",df['object_entity'][i])
        except:
            if find in df['object_entity'][i].split('\", \'start_idx')[0].split('word\': \"')[1]:
                print(find,i,": ",df['object_entity'][i])


for ot in bracket:
    find_sc(train_df,ot)

( 3331 :  {'word': '샤르자(Sharjah) 대학', 'start_idx': 16, 'end_idx': 30, 'type': 'ORG'}
( 4114 :  {'word': 'f(x)', 'start_idx': 13, 'end_idx': 16, 'type': 'ORG'}
( 6310 :  {'word': 'f(x)', 'start_idx': 25, 'end_idx': 28, 'type': 'ORG'}
( 9158 :  {'word': 'f(x)', 'start_idx': 35, 'end_idx': 38, 'type': 'ORG'}
( 9247 :  {'word': '현대로템(주)', 'start_idx': 3, 'end_idx': 9, 'type': 'ORG'}
( 9520 :  {'word': 'f(x)', 'start_idx': 28, 'end_idx': 31, 'type': 'ORG'}
( 11244 :  {'word': '(주)마인버스', 'start_idx': 73, 'end_idx': 79, 'type': 'ORG'}
( 13977 :  {'word': 'f(x)', 'start_idx': 50, 'end_idx': 53, 'type': 'ORG'}
( 14391 :  {'word': '아소카라자(Devanampiya Piyadasi Asokaraja)', 'start_idx': 55, 'end_idx': 91, 'type': 'PER'}
( 16644 :  {'word': 'E미래에셋증권(주)', 'start_idx': 13, 'end_idx': 22, 'type': 'ORG'}
( 17725 :  {'word': 'f(x)', 'start_idx': 8, 'end_idx': 11, 'type': 'ORG'}
( 18659 :  {'word': 'f(x)', 'start_idx': 36, 'end_idx': 39, 'type': 'ORG'}
( 20370 :  {'word': 'f(x)', 'start_idx': 25, 'end_idx

( 29446 :  {'word': '태조(太祖)', 'start_idx': 4, 'end_idx': 9, 'type': 'PER'}
( 30185 :  {'word': '노다(野田) 대감', 'start_idx': 41, 'end_idx': 49, 'type': 'PER'}
( 30331 :  {'word': '절강(浙江)', 'start_idx': 23, 'end_idx': 28, 'type': 'LOC'}
( 31449 :  {'word': '사카이 나카(酒井ナカ)', 'start_idx': 50, 'end_idx': 61, 'type': 'PER'}
( 31571 :  {'word': '현종(顯宗)', 'start_idx': 0, 'end_idx': 5, 'type': 'PER'}
( 31865 :  {'word': '김용수(金龍樹)', 'start_idx': 10, 'end_idx': 17, 'type': 'PER'}
( 31892 :  {'word': '김춘추(金春秋)', 'start_idx': 39, 'end_idx': 46, 'type': 'PER'}
( 31894 :  {'word': '성주(星州)', 'start_idx': 4, 'end_idx': 9, 'type': 'LOC'}
( 32020 :  {'word': '소릉(昭陵)', 'start_idx': 0, 'end_idx': 5, 'type': 'POH'}
( 32089 :  {'word': '웅진(熊津)', 'start_idx': 32, 'end_idx': 37, 'type': 'LOC'}
( 32174 :  {'word': '문명황후(文明皇后)', 'start_idx': 20, 'end_idx': 29, 'type': 'PER'}
( 32176 :  {'word': '라이언스(Lions)', 'start_idx': 15, 'end_idx': 25, 'type': 'POH'}
)  6657 :  {'word': '동치(同治) 13년 12월 5일(1875년 1월 12일)', 'start_

In [170]:
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        if find in df['sentence'][i]:
            print(df['sentence'][i])


for ot in bracket[2:]:
    print('\n\n',ot,'--------------------------------------')
    find_sc(test_df,ot)
    



 《 --------------------------------------
미국 의회도서관 연방연구부가 1993년에 발간한 《페르시아만 국가에 관한 연구》(Persian Gulf States Country Studies)에서 저자들은 "쿠웨이트, 카타르, 아랍에미리트, 오만, 바레인을 비롯한 아랍권 국가들의 정부가 '페르시아만'이라는 용어를 공식적으로 거부하고 '아라비아만'이라는 용어를 사용한다는 사실을 인정하면서도 '페르시아만'을 사용하는 미국 지명위원회의 관행을 따르고 있다."고 판단했다.
2015년 10월 19일 자정, 트와이스는 음원과 뮤직 비디오를 동시 공개하였고, 20일 첫 번째 EP 앨범 《THE STORY BEGINS》의 발매와 동시에 악스코리아 에서 데뷔 쇼케이스 무대를 개최하였다.
류득공은 《청일통지》에 기록된 발해의 지리에 대해서도 인용하여, 용천부, 현덕부, 용원부, 남해부, 부여부, 심주, 개주, 부주, 삼로군, 홀한하, 상경성, 평양, 곽주, 모주성에 대해 서술하였다.
2001년, 그는 제작이 시작된 Key의 차기작 《클라나드》와 비주얼 아트 산하의 다른 브랜드들에서 작곡을 했는데, 거기에는 지금은 없어진 브랜드 Words에서 2002년에 발매된 《벚나무 아래에서》와 2003년 발매된 Bonbee!의 《ALMA~쭉 곁에서...~》가 있다.
영 형제는 1988년 《Blow Up Your Video》를 프로듀싱했던 형 조지 영, 해리 반다와 함께 밴드의 초기 음반, 1995년 《Ballbreaker》 (릭 루빈과 함께)를 공동 프로듀싱해 《Stiff Upper Lip》을 완성한 마이크 프레이저에게 눈을 돌렸다.
《한겨레》의 기획 보도 중 에스더가 주최하는 강연의 내용 중 한민족을 이스라엘의 12 부족 중 단 지파의 후손으로 보는 이른바 '한민족 선민론'이 포함되어 있다고 주장하였으나, 에스더는 해당 강연에서 한번도 '한민족 선민론'이 포함되지 않았다고 주장했다.
이 작품으로 배우 최진혁으로서 주목을 받았고 이후 《상속자들》, 《응급남녀

반면, 『매송론』(梅松論)에 묘사되어 있는, 다타라하마 전투(多々良浜の戦い) 당시에 전투에 임한 다카우지의 출정 모습이 본 초상와 흡사하고, 교토로 개선하는 다카우지가 이때의 모습을 화공에게 그리게 하였다는 기록도 남아 있어 역시 「기마무사상」이 아시카가 다카우지의 초상화가 맞지 않겠느냐는 의견도 있다.
간기를 확인할 수 없어 명확한 인쇄시기를 판단하기 어려우나 병진자의 사용, 인쇄상태, 판식, 사정전훈의 『자치통감강목』이 1436년 7월 29일부터 1438년 11월 사이에 편찬·간행되었다는 기록 등으로 판단해보면 1438년 11월 이전에 인쇄되었을 것으로 여겨진다.
사단장은 북한군이 그대로 남진할 경우 서부전선이 위급하게 될 것임을 직감하고, 제7연대장 임부택 중령에게 『장호원을 즉각 탈취하라.』고 명령하였다.
그 후 미쓰쿠니는 아버지 요리후사의 죽음으로 인해 가업을 상속하였고, 공무 수행으로 인해 바빠지는 바람에 편수 사업으로부터는 멀어지게 되었지만, 한편으로 막부에서는 간분 2년(1662년)에 하야시 가호(林鵞峰)에게 명해 편년체의 사서 『본조통감(本朝通鑑)』의 편찬을 개시하였고, 미쓰쿠니는 하야시 가보를 번저에 초대하여 면담하며, 편찬 방침이나 정통성 문제에 대해 질의하였다.
지그문트 프로이트(Sigmund Freud)는 자기애 방어에 특별히 주목하지 않았지만, 『자기애론(On Narcissim)』에서, 문학작품에서 묘사되는 것처럼 범죄자나 유머 작가들이 자아를 축소시키는 것은 무엇이든 자아로부터 떨어뜨리는 자기애적 성향 지속으로 우리의 관심을 어떻게 강요하는지를 언급하였다.
블라디미르 레닌 사후 이오시프 스탈린은 마르크스주의 변증법과 유물론을 국정 철학으로 교조화하기 위해 여러가지 유물론 서적을 저작했는데, 대표적으로 『볼셰비키당사』 제4장인 『변증법적 유물론』이 있다.


 』  --------------------------------------
『후한서』 동이전에 따르면 건무중원 2년(기원후 57년) 후한 광무제에게 왜노국(倭奴国)의 사신이 찾아

1604년부터 심당길 등은 조선식 가마를 만들어 구로몬[黑物]이라는 조선 분황사기 비슷한 것을 만들었다.
고조부 원안 이래 4대가 삼공에 오른[사세삼공, 四世三公] 명문 집안에서 원봉의 적자로 태어나 그 위세가 대단했다.
그러자 다윗은 [[요압]] 대신 요압의 어머니 [[스루야]]의 자매 [[나하스]]의 딸 [[아비가일]]과 혼인 관계인 [[이드라]]의 아들 [[아마사]]를 내세웠다.


 ]  --------------------------------------
③ 하나님[께서] 2대 슬픔을 갖고 계시는데 그 첫째가 아담이 타락하는 순간을 아시면서 간섭하시지 못하시고 바라만 볼 수밖에 없으셨던 슬픔, 둘째는 예수님께서 하나님의 뜻을 십자가에 돌아가시지 않고 살아서 이루셔야 하는데도 불구하고 인간의 불신으로 말미암아 예수님이 십자가에 못 박히시는 장면을 보시는 슬픔이었다.
파머는 그후 2011년 중반 미시건주 앤아버에서 촬영한 독립 로맨스 영화 《[[사랑과 전쟁 (2013년 영화)|사랑과 전쟁]》에 [[리엄 헴스워스]]와 같이 출연했다. 이 영화는 [[베트남 전쟁] 기간인 1960년대를 배경으로 했으며, 2013년 3월에 개봉되었다.
다윗이 마하나임에 이르렀을 때 [[암몬]] 사람들이 성읍 랍바에서 [[나하스]]의 아들 [[소비]]가 왔고 로 로데발에서는 [[암미엘]]의 아들 [[마길]]이 왔고 로글림에서는 길르앗 출신의 사람 [[바르실래]]가 왔다.
세계 2차 대전이 끝난 평시에 연합군 회원국 출신의 대표자들이 미국의 [[드와이트 D. 아이젠하워]] [[미국의 대통령|대통령]]을 축하하기 위해 리도 홀에서 무도회를 열렸다.
경기 남양주시(시장 조광한)가 9~10월 두달간 시의 핵심가치 공유를 위한 직원 교육과정인 [시즌3] '3·3·3 핵심인재 역량강화교육'을 시작했다.
고조부 원안 이래 4대가 삼공에 오른[사세삼공, 四世三公] 명문 집안에서 원봉의 적자로 태어나 그 위세가 대단했다.
그러자 다윗은 [[요압]] 대신 요압의 어머니 [[스루야]]의 자매 [[나

In [166]:
# subject, object 확인
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        try:
            if find in df['subject_entity'][i].split('\', \'start_idx')[0].split('word\': \'')[1]:
                print(find,i,": ",df['subject_entity'][i])
        except:
            print('!!!!!!error',find,i,": ",df['subject_entity'][i])


for ot in bracket:
    find_sc(test_df,ot)
    
print('\n----------------------------------------------------------\n')
# subject, object 확인
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        try:
            if find in df['object_entity'][i].split('\', \'start_idx')[0].split('word\': \'')[1]:
                print(find,i,": ",df['object_entity'][i])
        except:
            try:
                if find in df['object_entity'][i].split('\", \'start_idx')[0].split('word\': \"')[1]:
                    print(find,i,": ",df['object_entity'][i])
            except:
                print('!!error!!!!!!!',find,i,": ",df['object_entity'][i])


for ot in bracket:
    find_sc(test_df,ot)

( 696 :  {'word': '(사)인천계양모범운전자회', 'start_idx': 50, 'end_idx': 62, 'type': 'ORG'}
( 2898 :  {'word': '(사)광주민주화운동기념사업회', 'start_idx': 62, 'end_idx': 76, 'type': 'ORG'}
( 3477 :  {'word': '(사)빛남영화공동체', 'start_idx': 15, 'end_idx': 24, 'type': 'ORG'}
( 3686 :  {'word': '글로벌 세아(주)', 'start_idx': 67, 'end_idx': 75, 'type': 'ORG'}
( 4122 :  {'word': '(사)한국드론산업협회', 'start_idx': 0, 'end_idx': 10, 'type': 'ORG'}
( 5660 :  {'word': '(사)한국전통민화연구소', 'start_idx': 0, 'end_idx': 11, 'type': 'ORG'}
( 6499 :  {'word': '(재)광양시어린이보육재단', 'start_idx': 4, 'end_idx': 16, 'type': 'ORG'}
( 6829 :  {'word': '(재)광양시어린이보육재단', 'start_idx': 4, 'end_idx': 16, 'type': 'ORG'}
( 7474 :  {'word': '(사)장보고글로벌재단', 'start_idx': 13, 'end_idx': 23, 'type': 'ORG'}
( 7503 :  {'word': '(재)전라남도환경산업진흥원', 'start_idx': 0, 'end_idx': 13, 'type': 'ORG'}
( 7558 :  {'word': '한국동서발전(주)', 'start_idx': 0, 'end_idx': 8, 'type': 'ORG'}
〈 5262 :  {'word': '〈창조〉', 'start_idx': 6, 'end_idx': 9, 'type': 'ORG'}
〉 5262 :  {'word': '〈창조〉', 'start_id

 ## UNK 
 특수문자중 unk로 변환되는 것이 붙은 단어는 단어 전체가 unk로 바뀐다

In [216]:
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        if find in df['sentence'][i]:
            print(df['sentence'][i])
            print(df['subject_entity'][i])
            print(df['object_entity'][i])
            try:
                print('label:',df['label'][i])
            except:
                pass
            print('\n')


for ot in unk_list:
    print('\n\n\n',ot,'#######################################')
    print('----train')
    find_sc(train_df,ot)
    print('----test')
    find_sc(test_df,ot)
    




 ↔ #######################################
----train
(KIA 내야수 그, 투수 이동현, 외야수 김경언 ↔ 한화 투수 안영명, 박성호, 외야수 김다원). 이 트레이드 상대 중 안영명은 이듬해 이범호의 보상 선수로 팀에 복귀했다.
{'word': '김경언', 'start_idx': 24, 'end_idx': 26, 'type': 'PER'}
{'word': '외야수', 'start_idx': 20, 'end_idx': 22, 'type': 'POH'}
label: per:title


(KIA 내야수 그, 투수 이동현, 외야수 김경언 ↔ 한화 투수 안영명, 박성호, 외야수 김다원). 이 트레이드 상대 중 안영명은 이듬해 이범호의 보상 선수로 팀에 복귀했다.
{'word': '김다원', 'start_idx': 50, 'end_idx': 52, 'type': 'PER'}
{'word': '외야수', 'start_idx': 20, 'end_idx': 22, 'type': 'POH'}
label: per:title


----test



 – #######################################
----train
미하일 8세 팔레올로고스(1223년 – 1282년 12월 11일)는 비잔티움 제국의 마지막 왕조인 팔레올로고스 왕조의 황제(재위기간:1261년~1282년)이다.
{'word': '팔레올로고스 왕조', 'start_idx': 55, 'end_idx': 63, 'type': 'ORG'}
{'word': '미하일 8세', 'start_idx': 0, 'end_idx': 5, 'type': 'PER'}
label: org:top_members/employees


앤더슨과 기네스는 오스트리아 작가 슈테판 츠바이크의 특히 중편 소설 《Twenty-Four Hours in the Life of a Woman》(1939)와 《The Post Office Girl》(1982), 그리고 그의 자서전 《The W

1985년 Lübeck(뤼벡) 콘서트에는 약700명의 오이! 스킨헤드들과 소수의 네오나치들이 운집하였는데, 케빈은 Deutschlandlied의 한 구절을 'schwarz-rot-gold(현재독일 깃발 색)' 대신 'schwarz–weiß-rot(흑백적-나치독일의 깃발)' 이라고 바꾸어 불렀다.
{'word': '네오나치', 'start_idx': 45, 'end_idx': 48, 'type': 'ORG'}
{'word': '1985년', 'start_idx': 0, 'end_idx': 4, 'type': 'DAT'}
label: 100


자기 팀은 폴란드에 0 – 1로 지고 있고 세네갈이 동점골을 넣으면 모든 게 말짱 도루묵으로 끝나는 상황에서 콜롬비아가 실점하지 않기를 간절히 기도하며 열심히 볼 돌리고 시간 지연하는 행위가 과연 전략이라고 넘길 만한 것인지 이해할 수가 없는 행동이다.
{'word': '세네갈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}
{'word': '0 – 1', 'start_idx': 11, 'end_idx': 15, 'type': 'NOH'}
label: 100


아버지 새뮤얼 애덤스 1세(Samuel Adams, Sr., 1689–1748)는 부유한 상인이면서 교회 집사이었다.
{'word': '새뮤얼 애덤스 1세', 'start_idx': 4, 'end_idx': 13, 'type': 'PER'}
{'word': '1689', 'start_idx': 34, 'end_idx': 37, 'type': 'DAT'}
label: 100


송선재 연구원은 “외형 축소로 인한 역 레버리지 효과가 발생하면서 영업이익률이 하락했고 특히 중국 법인의 적자 폭이 –3억원에서 -28억원으로 확대됐다”며 “순이익은 전년 상반기 42억원에서 올해 상반기 81억원으로 증가했는데, 전년 발생했던 잡손실이 제거됐기 때문”이라고 설명했다.
{'word': '송선재', 'start_idx': 0, 'end_idx

----test



 ☎ #######################################
----train
기타 소상공인 특례보증에 대한 궁금한 사항은 경기신용보증재단 고양지점(☎1577-5900)이나 인터넷(고양시청 콜센터(☎031-909-9000)(☎031-909-9000)로 문의하면 된다.
{'word': '고양시', 'start_idx': 57, 'end_idx': 59, 'type': 'ORG'}
{'word': '031-909-9000', 'start_idx': 67, 'end_idx': 78, 'type': 'NOH'}
label: no_relation


기타 소상공인 특례보증에 대한 궁금한 사항은 경기신용보증재단 고양지점(☎1577-5900)이나 인터넷(고양시청 콜센터(☎031-909-9000)(☎031-909-9000)로 문의하면 된다.
{'word': '고양시', 'start_idx': 57, 'end_idx': 59, 'type': 'ORG'}
{'word': '031', 'start_idx': 67, 'end_idx': 69, 'type': 'NOH'}
label: no_relation


자세한 내용은 파주시 홈페이지(파주기업SOS넷(paju.giupsos.or.kr)(paju.giupsos.or.kr), 중소기업 지원정보 포털사이트인 이지비즈(확인할 수 있으며 기업지원과 기업SOS팀(☎031-940-4533)(☎031-940-4533)으로 문의하면 된다.
{'word': '파주시', 'start_idx': 8, 'end_idx': 10, 'type': 'ORG'}
{'word': '031-940-4533', 'start_idx': 113, 'end_idx': 124, 'type': 'POH'}
label: no_relation


자세한 사항은 여수시립도서관 홈페이지를 참고하거나 현암도서관(☎659-4798)으로 문의하면 된다.
{'word': '여수시', 'start_idx': 8, 'end_idx': 10, 'type':

2월 9일 김일성은 친일 잔재 청산, 토지 개혁, 주요 산업 발전 지원, 중소 상공업 육성, 노동운동 지원, 민주주의 교육⋅문화 정책 수립 등 「북조선 임시 인민 위원회 당면 과업」 11개조를 제시하였다.
{'word': '북조선 임시 인민 위원회', 'start_idx': 81, 'end_idx': 93, 'type': 'ORG'}
{'word': '김일성', 'start_idx': 6, 'end_idx': 8, 'type': 'PER'}
label: no_relation


대산농촌재단은 교보생명 창립자 대산 신용호 선생의 뜻을 받들어 농촌사회 복리증진 도모를 목적으로 1991년 10월 25일 설립된 우리나라 최초의 농업⋅농촌지원 공익재단이다.
{'word': '교보생명', 'start_idx': 8, 'end_idx': 11, 'type': 'ORG'}
{'word': '신용호', 'start_idx': 20, 'end_idx': 22, 'type': 'PER'}
label: org:founded_by


----test
고(故) 김수연 선생은 학성강당을 설립하고 70여년의 세월 동안 1만여명의 제자를 배출⋅육성했으며 이러한 공로로 2003년 KBS 전북의 어른상, 2005년 국무총리상, 2013년 국민훈장 석류장 등을 수상했다.
{'word': '학성강당', 'start_idx': 13, 'end_idx': 16, 'type': 'ORG'}
{'word': '김수연', 'start_idx': 5, 'end_idx': 7, 'type': 'PER'}
label: 100





 ⟪ #######################################
----train
그 후 쿤은 양자 역학의 형성 과정에 대해 막스 플랑크라는 물리학자를 중심으로 연구를 수행했고, 그 결과는 양자 물리학의 발달과정의 주요 논문을 모은 ⟪양자물리학의 역사에 대한 자료집⟫("Sources for the History of Quantum Physics", 1967)'과 ⟪흑체

공무원연금공단(이사장 정남준)은 21일 전북 전주시 꽃밭정이노인복지관에서 ˹찾아가는 사랑나눔 따뜻한 겨울나기 문화공연˼을 개최했다.
{'word': '공무원연금공단', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}
{'word': '정남준', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
label: org:top_members/employees


공무원연금공단(이사장 정남준)은 12일 전북 완주시 삼례 전통시장에서 동절기 ˹재래시장 화재점검 및 안전예방˼ 봉사활동을 실시했다.
{'word': '공무원연금공단', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}
{'word': '정남준', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
label: org:top_members/employees


----test



 ˼ #######################################
----train
공무원연금공단(이사장 정남준)은 21일 전북 전주시 꽃밭정이노인복지관에서 ˹찾아가는 사랑나눔 따뜻한 겨울나기 문화공연˼을 개최했다.
{'word': '공무원연금공단', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}
{'word': '정남준', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
label: org:top_members/employees


공무원연금공단(이사장 정남준)은 12일 전북 완주시 삼례 전통시장에서 동절기 ˹재래시장 화재점검 및 안전예방˼ 봉사활동을 실시했다.
{'word': '공무원연금공단', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}
{'word': '정남준', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
label: org:top_members

자히르 알딘 무함마드 바부르(우르두어: ظہیرالدین محمد بابر, 힌두어: ज़हीरुद्दीन मुहम्मद बाबर, 1483년 2월 23일; 율리우스력으로는 2월 14일 ~ 1531년 1월 5일; 율리우스력 1530년 12월 26일)는 안디잔(지금의 우즈베키스탄의 한 지방) 출신의 모험가로, 여러 차례 실패를 경험했으나 1504년 결국 아프가니스탄의 카불에서 왕국을 세우는데 성공했다.
{'word': '바부르', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
{'word': '아프가니스탄의 카불', 'start_idx': 198, 'end_idx': 207, 'type': 'LOC'}
label: per:place_of_residence


자히르 알딘 무함마드 바부르(우르두어: ظہیرالدین محمد بابر, 힌두어: ज़हीरुद्दीन मुहम्मद बाबर, 1483년 2월 23일; 율리우스력으로는 2월 14일 ~ 1531년 1월 5일; 율리우스력 1530년 12월 26일)는 안디잔(지금의 우즈베키스탄의 한 지방) 출신의 모험가로, 여러 차례 실패를 경험했으나 1504년 결국 아프가니스탄의 카불에서 왕국을 세우는데 성공했다.
{'word': '바부르', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
{'word': '1530년 12월 26일', 'start_idx': 125, 'end_idx': 137, 'type': 'DAT'}
label: per:date_of_death


----test



 ु #######################################
----train
자히르 알딘 무함마드 바부르(우르두어: ظہیرالدین محمد بابر, 힌두어: ज़हीरुद्दीन मुहम्मद बाबर, 1483년 2월 23일; 율리우스력으로는 2월 14일 ~ 1531년 1월 5일; 율리우스력 1530년 12월 26일)는 안디잔(지금의 우즈